In [1]:
from mc_openapi.doml_mc.intermediate_model.metamodel import parse_metamodel, parse_inverse_associations
import yaml
with open("../assets/doml_meta.yaml") as mmf:
    mmdoc = yaml.load(mmf, yaml.Loader)
mm = parse_metamodel(mmdoc)
inv_assoc = parse_inverse_associations(mmdoc)

In [2]:
import prettyprinter as pp
from prettyprinter.prettyprinter import IMPLICIT_MODULES
pp.install_extras(include=['dataclasses'])

In [3]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.metamodel')
pp.pprint(mm["application_SoftwareComponent"])

mc_openapi.doml_mc.intermediate_model.metamodel.DOMLClass(
    name='application_SoftwareComponent',
    superclass='application_ApplicationComponent',
    attributes={
        'isPersistent': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='isPersistent',
            type='Boolean',
            multiplicity=('1', '1'),
            default=False
        ),
        'licenseCost': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='licenseCost',
            type='String',
            multiplicity=('0', '1'),
            default=None
        ),
        'configFile': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAttribute(
            name='configFile',
            type='String',
            multiplicity=('0', '1'),
            default=None
        )
    },
    associations={
        'exposedInterfaces': mc_openapi.doml_mc.intermediate_model.metamodel.DOMLAssociation(
            name='exposedInterfaces',
            class_

### Load the DOML document here:

In [4]:
from mc_openapi.doml_mc.xmi_parser.doml_model import parse_doml_model
doml_document_path = "/home/michele/Repositories/piacere/DOML-case-study/faas/doml/faas.domlx"
# doml_document_path = "../../tests/doml/nginx-openstack_v2_wrong.domlx"
with open(doml_document_path, "rb") as xmif:
    doc = xmif.read()

im = parse_doml_model(doc, mm)
print(im)

{'source_code': DOMLElement(name='source_code', class_='commons_SProperty', attributes={'commons_Property::key': 'source_code', 'commons_SProperty::value': 'path/lambda/function/image_resize.py'}, associations={}), 'storage_interface': DOMLElement(name='storage_interface', class_='application_SoftwareInterface', attributes={'commons_DOMLElement::name': 'storage_interface'}, associations={}), 'bucket_in': DOMLElement(name='bucket_in', class_='application_SoftwareInterface', attributes={'commons_DOMLElement::name': 'bucket_in'}, associations={}), 'bucket_out': DOMLElement(name='bucket_out', class_='application_SoftwareInterface', attributes={'commons_DOMLElement::name': 'bucket_out'}, associations={}), 'handle_image': DOMLElement(name='handle_image', class_='application_SoftwareInterface', attributes={'commons_DOMLElement::name': 'handle_image'}, associations={}), 'image_resize': DOMLElement(name='image_resize', class_='application_SoftwareComponent', attributes={'application_SoftwareCom

Attribute cidr of multiplicity > 1 not supported yet.
Attribute cidr of multiplicity > 1 not supported yet.
Attribute cidr of multiplicity > 1 not supported yet.
Attribute cidr of multiplicity > 1 not supported yet.


In [7]:
from mc_openapi.doml_mc.intermediate_model.doml_element import reciprocate_inverse_associations
reciprocate_inverse_associations(im, inv_assoc)

In [8]:
IMPLICIT_MODULES.add('doml_mc.intermediate_model.doml_element')
pp.pprint(im)

{
    'source_code': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='source_code',
        class_='commons_SProperty',
        attributes={
            'commons_Property::key': 'source_code',
            'commons_SProperty::value': 'path/lambda/function/image_resize.py'
        },
        associations={}
    ),
    'storage_interface': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='storage_interface',
        class_='application_SoftwareInterface',
        attributes={'commons_DOMLElement::name': 'storage_interface'},
        associations={}
    ),
    'bucket_in': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='bucket_in',
        class_='application_SoftwareInterface',
        attributes={'commons_DOMLElement::name': 'bucket_in'},
        associations={}
    ),
    'bucket_out': mc_openapi.doml_mc.intermediate_model.doml_element.DOMLElement(
        name='bucket_out',
        class_='applicatio

In [9]:
unbound_elems_n = 0
unbound_elems = [f"unbound{i}" for i in range(unbound_elems_n)]

In [10]:
from z3 import Solver

from mc_openapi.doml_mc.z3encoding.metamodel_encoding import (
    def_association_rel_and_assert_constraints,
    def_attribute_rel_and_assert_constraints,
    mk_association_sort_dict,
    mk_attribute_sort_dict,
    mk_class_sort_dict
)
from mc_openapi.doml_mc.z3encoding.im_encoding import (
    assert_im_associations_q,
    assert_im_attributes,
    def_elem_class_f_and_assert_classes,
    mk_elem_sort_dict,
    mk_stringsym_sort_dict
)
from mc_openapi.doml_mc.z3encoding.utils import mk_adata_sort

solver = Solver()

class_sort, class_ = mk_class_sort_dict(mm)
assoc_sort, assoc = mk_association_sort_dict(mm)
attr_sort, attr = mk_attribute_sort_dict(mm)
elem_sort, elem = mk_elem_sort_dict(im, unbound_elems)
ss_sort, ss = mk_stringsym_sort_dict(im, mm)
AData = mk_adata_sort(ss_sort)
elem_class_f = def_elem_class_f_and_assert_classes(
    im,
    solver,
    elem_sort,
    elem,
    class_sort,
    class_
)
attr_rel = def_attribute_rel_and_assert_constraints(
    mm,
    solver,
    attr_sort,
    attr,
    class_,
    elem_class_f,
    elem_sort,
    AData,
    ss
)
assert_im_attributes(
    attr_rel,
    solver,
    im,
    mm,
    elem,
    attr_sort,
    attr,
    AData,
    ss
)
assoc_rel = def_association_rel_and_assert_constraints(
    mm,
    solver,
    assoc_sort,
    assoc,
    class_,
    elem_class_f,
    elem_sort,
    inv_assoc
)
assert_im_associations_q(
    assoc_rel,
    solver,
    {k: v for k, v in im.items() if k not in unbound_elems},
    elem,
    assoc_sort,
    assoc,
)

In [11]:
# solver.push()

### Metamodel statistics

In [12]:
from mc_openapi.doml_mc.intermediate_model.metamodel import get_subclasses_dict

print("Number of classes:", len(mm))
print(
    "Number of tuples in the subclass relation:",
    sum(len(s) for s in get_subclasses_dict(mm).values())
)
n_attrs = sum(len(c.attributes) for c in mm.values())
print("Number of attributes:", n_attrs)
print("Number of attribute assertions:", 3 * n_attrs)
n_assocs = sum(len(c.associations) for c in mm.values())
print("Number of associations:", n_assocs)
print("Number of associations assertions:", 3 * n_assocs + len(inv_assoc))



Number of classes: 51
Number of tuples in the subclass relation: 133
Number of attributes: 47
Number of attribute assertions: 141
Number of associations: 56
Number of associations assertions: 174


### DOML model statistics

In [13]:
n_elems = len(im)
print("Number of elements:", n_elems)
n_attrs_im = sum(len(e.attributes) for e in im.values())
print("Number of attributes (IM):", n_attrs_im)
n_assocs_im = sum(len(a) for e in im.values() for a in e.associations.values())
print("Number of associations (IM):", n_assocs_im)
print("Number of string symbols:", len(ss))
print("Number of IM attribute assertions:", n_elems)
print("Number of IM association assertions:", n_elems ** 2)

Number of elements: 50
Number of attributes (IM): 93
Number of associations (IM): 81
Number of string symbols: 65
Number of IM attribute assertions: 50
Number of IM association assertions: 2500


In [14]:
# solver.check()

In [15]:
# solver.unsat_core()

In [16]:
# solver.statistics()

In [17]:
from z3 import Consts, ForAll, Exists, Implies, And, Or

In [19]:
# All VMs have a network interface.
vm, iface = Consts("vm iface", elem_sort)
vmIfaceAssertion = ForAll(
    [vm],
    Implies(
        elem_class_f(vm) == class_["infrastructure_VirtualMachine"],
        Exists(
            [iface],
            assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], iface),
        )
    )
)
solver.assert_and_track(vmIfaceAssertion, "vm_iface")

In [18]:
# All software packages can see the interfaces they need through a common network.
asc_consumer, asc_exposer, siface, net, net_iface, cn, vm, deployment, dc = Consts(
    "asc_consumer asc_exposer siface net net_iface cn vm deployment dc", elem_sort
)
assn = ForAll(
    [asc_consumer, asc_exposer, siface],
    Implies(
        And(
            assoc_rel(asc_consumer, assoc["application_SoftwareComponent::exposedInterfaces"], siface),
            assoc_rel(asc_exposer, assoc["application_SoftwareComponent::consumedInterfaces"], siface),
        ),
        Exists(
            [net],
            And(
                Or(
                    Exists(
                        [cn, deployment, net_iface],
                        And(  # asc_consumer is deployed on a component with an interface in network n
                            assoc_rel(deployment, assoc["commons_Deployment::component"], asc_consumer),
                            assoc_rel(deployment, assoc["commons_Deployment::node"], cn),
                            assoc_rel(cn, assoc["infrastructure_ComputingNode::ifaces"], net_iface),
                            assoc_rel(net_iface, assoc["infrastructure_NetworkInterface::belongsTo"], net),
                        ),
                    ),
                    Exists(  # asc_consumer is deployed on a container hosting a VM with an interface in network n
                        [cn, deployment, vm, net_iface],
                        And(
                            assoc_rel(deployment, assoc["commons_Deployment::component"], asc_consumer),
                            assoc_rel(deployment, assoc["commons_Deployment::node"], cn),
                            assoc_rel(cn, assoc["infrastructure_Container::hosts"], vm),
                            assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], net_iface),
                            assoc_rel(net_iface, assoc["infrastructure_NetworkInterface::belongsTo"], net),
                        ),
                    ),
                ),
                Or(
                    Exists(
                        [cn, deployment, net_iface],
                        And(  # asc_exposer is deployed on a component with an interface in network n
                            assoc_rel(deployment, assoc["commons_Deployment::component"], asc_exposer),
                            assoc_rel(deployment, assoc["commons_Deployment::node"], cn),
                            assoc_rel(cn, assoc["infrastructure_ComputingNode::ifaces"], net_iface),
                            assoc_rel(net_iface, assoc["infrastructure_NetworkInterface::belongsTo"], net),
                        ),
                    ),
                    Exists(  # asc_exposer is deployed on a container hosting a VM with an interface in network n
                        [cn, deployment, vm, net_iface],
                        And(
                            assoc_rel(deployment, assoc["commons_Deployment::component"], asc_exposer),
                            assoc_rel(deployment, assoc["commons_Deployment::node"], cn),
                            assoc_rel(cn, assoc["infrastructure_Container::hosts"], vm),
                            assoc_rel(vm, assoc["infrastructure_ComputingNode::ifaces"], net_iface),
                            assoc_rel(net_iface, assoc["infrastructure_NetworkInterface::belongsTo"], net),
                        ),
                    ),
                ),
            ),
        ),
    ),
)
solver.assert_and_track(assn, "software_package_iface_net")

In [16]:
# SoftwareComponents have a source_code property
softwareComponent, prop = Consts("softwareComponent prop", elem_sort)
nginxSourceCode = ForAll(
    [softwareComponent],
    Implies(
        And(
            elem_class_f(softwareComponent) == class_["application_SoftwareComponent"],
            # attr_rel(softwareComponent, attr["commons_DOMLElement::name"], AData.ss(ss["web"]))
        ),
        Exists(
            [prop],
            And(
                elem_class_f(prop) == class_["commons_SProperty"],
                attr_rel(prop, attr["commons_Property::key"], AData.ss(ss["source_code"])),
                assoc_rel(softwareComponent, assoc["commons_DOMLElement::annotations"], prop)
            )
        )
    )
)
solver.assert_and_track(nginxSourceCode, "SoftwareComponent_source_code")

In [16]:
# There are no duplicated interfaces
def any_iface(elem, iface):
    ifaces_assocs = [
        "infrastructure_ComputingNode::ifaces",
        "infrastructure_Storage::ifaces",
        "infrastructure_FunctionAsAService::ifaces"
    ]
    return Or(*(assoc_rel(elem, assoc[assoc_name], iface) for assoc_name in ifaces_assocs))
e1, e2, ni = Consts("e1 e2 i", elem_sort)
assn = ForAll([e1, e2, ni],
    Implies(
        And(
            any_iface(e1, ni),
            any_iface(e2, ni)
        ),
        e1 == e2
    )
)
solver.assert_and_track(assn, "iface_uniq")

In [18]:
# All SoftwareComponents have been deployed somewhere
sc, deployment, ielem = Consts("sc deployment ielem", elem_sort)
assn = ForAll(
    [sc],
    Implies(
        elem_class_f(sc) == class_["application_SoftwareComponent"],
        Exists(
            [deployment, ielem],
            And(
                assoc_rel(deployment, assoc["commons_Deployment::component"], sc),
                assoc_rel(deployment, assoc["commons_Deployment::node"], ielem)
            )
        )
    )
)
solver.assert_and_track(assn, "all_SoftwareComponents_deployed")

In [16]:
# All abstract infrastructure elements are mapped to an element in the active concretization
def checkOneClass(ielem, concr, provider, celem, ielemClass, providerAssoc, celemAssoc):
    return Implies(
                elem_class_f(ielem) == class_[ielemClass],
                Exists(
                    [provider, celem],
                    And(
                        assoc_rel(concr, assoc["concrete_ConcreteInfrastructure::providers"], provider),
                        assoc_rel(provider, assoc[providerAssoc], celem),
                        assoc_rel(celem, assoc[celemAssoc], ielem)
                    )
                )
            )

ielem, concr, provider, celem = Consts("ielem, concr, provider, celem", elem_sort)
assn = Exists(
    [concr],
    And(
        elem_class_f(concr) == class_["concrete_ConcreteInfrastructure"],
        ForAll(
            [ielem],
            And(
                checkOneClass(
                    ielem, concr, provider, celem,
                    "infrastructure_VirtualMachine",
                    "concrete_RuntimeProvider::vms",
                    "concrete_VirtualMachine::maps"
                ),
                checkOneClass(
                    ielem, concr, provider, celem,
                    "infrastructure_Network",
                    "concrete_RuntimeProvider::networks",
                    "concrete_Network::maps"
                ),
                checkOneClass(
                    ielem, concr, provider, celem,
                    "infrastructure_Storage",
                    "concrete_RuntimeProvider::storages",
                    "concrete_Storage::maps"
                ),
                checkOneClass(
                    ielem, concr, provider, celem,
                    "infrastructure_FunctionAsAService",
                    "concrete_RuntimeProvider::faas",
                    "concrete_FunctionAsAService::maps"
                ),
            )
        )
    )
)
solver.assert_and_track(assn, "all_infrastructure_elements_deployed")

In [18]:
# The "web" component has access to the Internet through https
web, deployment, ielem1, ielem2, iface, sg, rule = Consts("web deployment ielem1 ielem2 iface sg rule", elem_sort)
assn = ForAll(
    [web],
    Implies(
        attr_rel(web, attr["commons_DOMLElement::name"], AData.ss(ss["web"])),
        Exists(  # web is deployed on a container hosting a VM with an interface in network n
            [deployment, ielem1, ielem2, iface, rule],
            And(
                assoc_rel(deployment, assoc["commons_Deployment::component"], web),
                assoc_rel(deployment, assoc["commons_Deployment::node"], ielem1),
                assoc_rel(ielem1, assoc["infrastructure_Container::hosts"], ielem2),
                assoc_rel(ielem2, assoc["infrastructure_ComputingNode::ifaces"], iface),
                assoc_rel(iface, assoc["infrastructure_NetworkInterface::associated"], sg),
                assoc_rel(sg, assoc["infrastructure_SecurityGroup::rules"], rule),
                attr_rel(rule, attr["infrastructure_Rule::fromPort"], AData.int(443)),
                attr_rel(rule, attr["infrastructure_Rule::toPort"], AData.int(443)),
                attr_rel(rule, attr["infrastructure_Rule::kind"], AData.ss(ss["INGRESS"]))
            ),
        )
    )
)
solver.assert_and_track(assn, "web_has_https")

In [20]:
solver.check()

sat

In [21]:
solver.unsat_core()

[elem_class source_code commons_SProperty,
 elem_class storage_interface application_SoftwareInterface,
 elem_class bucket_in application_SoftwareInterface,
 elem_class bucket_out application_SoftwareInterface,
 elem_class handle_image application_SoftwareInterface,
 elem_class image_resize application_SoftwareComponent,
 elem_class s1 application_SaaS,
 elem_class s2 application_SaaS,
 elem_class notification application_SoftwareComponent,
 elem_class web application_SoftwareComponent,
 elem_class ImageResizeApp application_ApplicationLayer,
 elem_class vpc_subnet infrastructure_Subnet,
 elem_class vpc infrastructure_VPC,
 elem_class i1 infrastructure_NetworkInterface,
 elem_class __generated_name__0 infrastructure_Location,
 elem_class v_img infrastructure_VMImage,
 elem_class ssh_key infrastructure_KeyPair,
 elem_class vm1 infrastructure_VirtualMachine,
 elem_class icmp infrastructure_Rule,
 elem_class http infrastructure_Rule,
 elem_class https infrastructure_Rule,
 elem_class ssh infrastructure_Rule,
 elem_class sg infrastructure_SecurityGroup,
 elem_class f infrastructure_FunctionAsAService,
 elem_class c infrastructure_Container,
 elem_class c_img infrastructure_ContainerImage,
 elem_class st1 infrastructure_Storage,
 elem_class st2 infrastructure_Storage,
 elem_class infra infrastructure_InfrastructureLayer,
 elem_class lambda_role_name commons_SProperty,
 elem_class lambda_runtime commons_SProperty,
 elem_class lambda_handler commons_SProperty,
 elem_class lambda_timeout commons_IProperty,
 elem_class lambda_memory commons_IProperty,
 elem_class concrete_f concrete_FunctionAsAService,
 elem_class concrete_net concrete_Network,
 elem_class bucket_name commons_SProperty,
 elem_class s3_bucket_in concrete_Storage,
 elem_class s3_bucket_out concrete_Storage,
 elem_class instance_type commons_SProperty,
 elem_class ssh_key_name commons_SProperty,
 elem_class ec2_role_name commons_SProperty,
 elem_class concrete_vm concrete_VirtualMachine,
 elem_class aws concrete_RuntimeProvider,
 elem_class con_infra concrete_ConcreteInfrastructure,
 association_st_classes commons_Deployment::source,
 association_st_classes commons_Deployment::target,
 associations image_resize handle_image,
 associations notification handle_image,
 software_package_iface_net]

In [20]:
solver.statistics().memory

66.31

In [21]:
m = solver.model()
print(m)

[attribute_st_types infrastructure_Rule::toPort = True,
 associations sg out_all = True,
 associations i1 http = True,
 associations ag http = True,
 associations vm_flavor con_infra = True,
 associations openstack concrete_vm_image = True,
 attribute_mult_ub commons_BProperty::value = True,
 associations v_img name = True,
 attribute_mult_lb infrastructure_Rule::kind = True,
 associations vm1 name = True,
 attribute_st_types commons_DOMLElement::name = True,
 association_mult_ub concrete_FunctionAsAService::maps = True,
 associations concrete_vm vm_key_name = True,
 associations app openstack = True,
 association_inverse infrastructure_Container::generatedFrom infrastructure_ContainerImage::generatedContainers = True,
 associations app concrete_vm = True,
 associations out_all v_img = True,
 association_mult_ub infrastructure_Container::generatedFrom = True,
 association_st_classes infrastructure_FunctionAsAService::ifaces = True,
 associations net1 ssh = True,
 associations con_infra

In [22]:
from itertools import product
for (e1n, e1), a, (e2n, e2) in product(elem.items(), assoc.values(), elem.items()):
    if (e1n in unbound_elems or e2n in unbound_elems) and m.eval(assoc_rel(e1, a, e2)):
        print(e1, a, e2)